In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

import scanpy as sc
from scipy import sparse
from scipy.special import softmax

import networkx as nx
import torch

In [6]:
import sys
sys.path.append('../')
import came
from came import pipeline, pp, pl

In [7]:
came_resdir = Path("../_temp/('Baron_human', 'Baron_mouse')-(07-15 23.57.51)")
dpair, model = came.load_dpair_and_model(came_resdir, ckpt=14)
predictor = came.Predictor.load(came_resdir / 'predictor.json')

TypeError: load_dpair_and_model() got an unexpected keyword argument 'ckpt'